# 콜백(Callback)
   
Model을 학습시킬 때 부가적으로 옵션을 넣어서 수행 할 수 있도록 도와주는 객체   
   
모델 학습시 사용하는 .fit() 함수에 callbacks라는 파라미터로 지정 가능  
   
딥러닝 모델이 과적합되기 시작하면 새로운 데이터에서의 예측 성능을 신뢰하기가 어려워짐   


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# data 정규화
x_train = x_train / x_train.max()
x_test = x_test / 255.

In [2]:
# model 생성
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [3]:
# model compile
model.compile(
    optimizer='adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

  # 모델 체크포인트(Model CheckPoint)   
    
  epoch 별로 Model의 가중치를 저장하는 역할   
  체크포인트에 대한 하이퍼파라미터 소개   
    
  - filepath : 체크포인트를 저장하는 경로를 지정   
  - monitor : 저장 시 기준이 되는 지표를 설정   
  - save_best_only : monitor 옵션을 기준으로 가장 높은 epoch만 저장할지, epoch 마다 저장할지(True,false)
  - verbose : epoch마다 저장 여부를 알려주는 로그 메세지를 출력할지 말지(1,0)   
  - save_weights_only : 가중치만 저장할지 말지(True, False)
    - 단, 확장자가 .ckpt인 경우에만 사용가능
    - Tensorflow 버전이 올라가면서 keras 확장자가 새로 생김
    - 사용하려면 버전을 낮춰야함

  추가적으로 checkpoint로 생성한 파일을 통해 다른 모델에 재학습이 가능

In [5]:
# 체크포인트 설정
checkPoint = keras.callbacks.ModelCheckpoint(
    filepath = 'checkPoint.keras', # 확장자명이 .ckpt -> keras로 바뀜
    # 이 파일을 통해 다른 모델에 재학습이 가능(중요)
    # 단, 불필요한 정보가 들어가있어서 파일이 무거운 편

    save_best_only= True,
    # True : monitor 되고 있는 값 기준으로 가장 뛰어난 epoch 모델이 저장
    # False : epoch 마다 모델이 filepath{epoch} 형태로 저장

    monitor = 'val_loss', # 모델을 저장할 때 기준이 되는 값 지정
    # 테스트 데이터셋을 기준으로 손실이 가장 적을 때 저장하려면? 'val_loss'
    # 학습용 데이터셋을 기준으로 손실이 가장 적을 때 저장하려면? 'loss'

    verbose=1 # epoch 마다 저장 여부를 알려주는 로그 메세지를 출력할지 말지
    # 1이면 모델 저장시마다 메세지 출력, 0이면 그냥 저장
)

In [6]:
model.fit(
    x_train, y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    callbacks=[checkPoint]
)

Epoch 1/10
1865/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6214 - loss: 2.7838
Epoch 1: val_loss improved from inf to 0.44731, saving model to checkPoint.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 14s 6ms/step - accuracy: 0.6222 - loss: 2.7742 - val_accuracy: 0.9025 - val_loss: 0.4473
Epoch 2/10
1873/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9103 - loss: 0.4149
Epoch 2: val_loss improved from 0.44731 to 0.24749, saving model to checkPoint.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 5ms/step - accuracy: 0.9103 - loss: 0.4148 - val_accuracy: 0.9445 - val_loss: 0.2475
Epoch 3/10
1871/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9459 - loss: 0.2255
Epoch 3: val_loss improved from 0.24749 to 0.20889, saving model to checkPoint.keras
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - accuracy: 0.9459 - loss: 0.2255 - val_accuracy: 0.9512 - val_loss: 0.2089
Epoch 4/10
1871/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9593 - loss: 0.1613
Epoch 4: val_loss improved from 0.208

In [7]:
# 체크포인트 적용 전 모델
loss, acc = model.evaluate(x_test, y_test)
print(loss, acc)

model.load_weights('checkPoint.keras')
# 체크포인트 적용 후 모델
loss, acc = model.evaluate(x_test, y_test)
print(loss, acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9689 - loss: 0.1453
0.12259825319051743 0.9735999703407288
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9697 - loss: 0.1342
0.11566745489835739 0.9731000065803528
